## Installing OpenAI library

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.5 MB/s eta 0:00:00


## Setting up ChatGPT API

In [ ]:
# Importing Operating SYstem library
import os 

In [ ]:
# Setting the environment variable named "OPENAI_API_KEY" to the OpenAI API Key
os.environ["OPENAI_API_KEY"] = "your-API-key"

In [ ]:
# Obtaining the api_key
api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
import openai
# Connecting to OpenAI API
openai.api_key = api_key

## Plot Generation using ChatGPT

In [ ]:
import pandas as pd

In [ ]:
# Loading the final processed dataset for plot generation
df = pd.read_csv('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/M_Dataset_0_2K')

In [ ]:
# Function to get the length of each summary 
def apply_len(x):
  return len(x.split())
df['len'] = df['Summary'].apply(apply_len)

There's a hidden 500-word/4,000-character limit in ChatGPT for each individual input and output.

In [ ]:
# Filtering the summaries whose length is less than 450 words in order to match the ChatGPT limit
df = df[df['len']<450]

In [ ]:
# Checking a sample (REMOVE THIS)
sample = df['Summary'].iloc[1]
sample

In [ ]:
# Different Instruction based prompts tested for plot generation
prompt1 = "Give a 1-2 line plot for the following summary: "
prompt2 = "Based on the given summary, generate a 1-2 line plot: "
prompt3 = "Generate a simple plain text 1-2 line plot for the given summary: " # Best
prompt4 = "In 1-2 lines, briefly explain the plot of the following summary: "
prompt5 = "In layman terms generate a 1-2 lines plot for the given summary: "

In [ ]:
# Function to send the request to ChatGPT and receive the generated plot
def get_chatgpt_response(prompt, sample):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", 
     "content": f"{prompt} {sample}"}
  ])
  return completion.choices[0].message

In [ ]:
# Add code to apply the function on the entire final dataset

In [ ]:
# REMOVE THIS 
print(get_chatgpt_response(prompt3, sample))

## Rating Estimation by ChatGPT

Move this code segment to the T5 training notebook later 

In [ ]:
# Loss to be added in the custom loss
def get_chatgpt_rating(prompt, sample):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", 
     "content": f"{prompt} {sample}"}
  ])
  return 10 - int(completion.choices[0].message)

In [ ]:
# Different prompts tested in order to generate a sensible rating 
prompt1 = "Provide me rating between 0 and 10 (without any explanation), where 0 is the best and 10 is the worst, for the following story summary: " 
prompt2 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response):"
prompt3 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response). The rating should be based on writing style, coherence and capture strength. Summary:"
prompt4 = "Assign a rating between 0 and 10 to the given artificial story summary. Only give rating as the response (no reasoning). The rating should be based on writing style, coherence, and capture strength. Summary:" # Best
prompt5 = "Provide me rating between 0 and 10 (without any explanation),  for the following story summary:" 

In [ ]:
# REMOVE THIS
print(get_chatgpt_rating(prompt4, sample))